In [13]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd, configparser

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

In [4]:
config = configparser.ConfigParser()
config.read('../config.cfg')
EMBEDDING_FILE=config['FILES']['GLOVE']
TRAIN_DATA_FILE=config['FILES']['TRAIN']
TEST_DATA_FILE=config['FILES']['TEST']

In [5]:
embed_size = 50 # how big is each word vector
max_features = 20000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a comment to use

In [6]:
train = pd.read_csv(TRAIN_DATA_FILE)
test = pd.read_csv(TEST_DATA_FILE)

list_sentences_train = train["comment_text"].fillna("_na_").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("_na_").values

In [7]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [8]:
def get_coefs(word,*arr): 
    return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE))

In [9]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std

/home/tiphaine/Fil_Rouge/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  """Entry point for launching an IPython kernel.


(0.020940498, 0.6441043)

In [10]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [11]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
model.fit(X_t, y, batch_size=32, epochs=2, validation_split=0.1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/2
143613/143613 [==============================] - 390s 3ms/step - loss: 0.0591 - acc: 0.9795 - val_loss: 0.0486 - val_acc: 0.9818
Epoch 2/2
143613/143613 [==============================] - 401s 3ms/step - loss: 0.0443 - acc: 0.9833 - val_loss: 0.0489 - val_acc: 0.9818


In [23]:
test_label = pd.read_csv(config['FILES']['LABEL'])
test_label_strip = test_label[test_label.toxic != -1]
yt = test_label_strip[list_classes].values

In [21]:
test_label_strip.index

Int64Index([     5,      7,     11,     13,     14,     16,     17,     19,
                21,     22,
            ...
            153142, 153144, 153145, 153147, 153149, 153150, 153151, 153154,
            153155, 153156],
           dtype='int64', length=63978)

In [24]:
model.evaluate(X_te[test_label_strip.index], yt, batch_size=1024)

63978/63978 [==============================] - 17s 259us/step


[0.0818848138119427, 0.9659128306436644]

In [37]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="sigmoid")(x)
model2 = Model(inputs=inp, outputs=x)
model2.compile(loss='hinge', optimizer='adam', metrics=['accuracy'])

In [38]:
model2.fit(X_t, y, batch_size=32, epochs=2, validation_split=0.1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/2
143613/143613 [==============================] - 406s 3ms/step - loss: 0.9635 - acc: 0.6612 - val_loss: 0.9627 - val_acc: 0.9940
Epoch 2/2
143613/143613 [==============================] - 507s 4ms/step - loss: 0.9634 - acc: 0.8966 - val_loss: 0.9627 - val_acc: 0.9940


In [39]:
model2.evaluate(X_te[test_label_strip.index], yt, batch_size=1024)

63978/63978 [==============================] - 24s 381us/step


[0.9622318247758406, 0.9975929227646407]